In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta ,date

import warnings
warnings.filterwarnings('ignore')

In [2]:
usertab = pd.read_csv("userTable.csv")
vistorlog = pd.read_csv("VisitorLogsData.csv")

In [3]:
usertab.head()

,UserID,Signup Date,User Segment
0,U133159,2018-04-14 07:01:16.202607+00:00,C
1,U129368,2017-12-02 09:38:41.584270+00:00,B
2,U109654,2013-03-19 11:38:55+00:00,B
3,U108998,2018-01-18 08:29:51.627954+00:00,C
4,U131393,2018-03-27 08:05:28.806800+00:00,B


In [4]:
vistorlog.head()

,webClientID,VisitDateTime,ProductID,UserID,Activity,Browser,OS,City,Country
0,WI10000050298,2018-05-07 04:28:45.970,pr100631,NaN,NaN,Chrome Mobile,Android,Chennai,India
1,WI10000025922,2018-05-13 07:26:04.964,pr100707,NaN,NaN,Chrome,Windows,NaN,Taiwan
2,WI100000204522,2018-05-11 11:43:42.832,pr100030,NaN,click,Chrome,windows,Gurgaon,India
3,WI10000011974,2018-05-13 15:20:23.436,Pr100192,NaN,CLICK,Chrome,Windows,,
4,WI100000441953,2018-05-08 20:44:25.238,Pr100762,NaN,click,Chrome,mac os x,Iselin,United States


In [5]:
print(usertab.shape)
print(vistorlog.shape)

(34050, 3)
(6588000, 9)


In [6]:
usertab.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34050 entries, 0 to 34049
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   UserID        34050 non-null  object
 1   Signup Date   34050 non-null  object
 2   User Segment  34050 non-null  object
dtypes: object(3)
memory usage: 798.2+ KB


In [7]:
vistorlog.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6588000 entries, 0 to 6587999
Data columns (total 9 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   webClientID    object
 1   VisitDateTime  object
 2   ProductID      object
 3   UserID         object
 4   Activity       object
 5   Browser        object
 6   OS             object
 7   City           object
 8   Country        object
dtypes: object(9)
memory usage: 452.4+ MB


In [8]:
vistorlog.isnull().sum()/vistorlog.shape[0]

webClientID      0.000000
VisitDateTime    0.100017
ProductID        0.080015
UserID           0.901230
Activity         0.135010
Browser          0.000000
OS               0.000000
City             0.328754
Country          0.060366
dtype: float64

In [9]:
vistorlog['VisitDateTime'].fillna(pd.to_datetime(pd.to_datetime('2018-5-28') - pd.Timedelta(days=21)),inplace=True) #choose 21 days before end date)

In [10]:
#droping nullds in 'UserID','VisitDateTime'
vistorlog.dropna(axis=0,subset=['UserID'], inplace=True)

In [11]:
#Droping City and Country
vistorlog.drop(['City','Country'],axis=1,inplace=True)
usertab.drop(['User Segment'],axis=1,inplace=True)

In [12]:
vistorlog.head()

,webClientID,VisitDateTime,ProductID,UserID,Activity,Browser,OS
14,WI100000112772,1526366895249000000,Pr100017,U106593,click,Chrome Mobile,Android
21,WI1000009977,1527058921790000000,Pr101008,U108297,NaN,Chrome Mobile,Android
23,WI100000223826,2018-05-10 06:28:53.391,Pr100241,U132443,NaN,Firefox,Windows
24,WI10000021998,2018-05-08 12:40:02.153,pr100495,U134616,click,Chrome,Windows
33,WI10000036281,2018-05-11 15:35:43.689,Pr100363,U130784,click,Chrome,Chrome OS


In [13]:
#Converting Unix date to normal date
vistorlog['VisitDateTime'] = (vistorlog.VisitDateTime.apply(lambda x : pd.to_datetime(int(x)) if '-' not in str(x) else pd.to_datetime(x) ))

In [14]:
#Filtering dataframe within provided date range 07-May-2018 to 27-May-2018.
vistorlog_filter =  vistorlog[(vistorlog.VisitDateTime >= '2018-5-7') & (vistorlog.VisitDateTime < '2018-5-28')]

In [15]:
#joining two tables

# vistorlog_filter.join(usertab, on ='UserID',how='inner' ,sort=True)
# df_inner = pd.merge(vistorlog_filter, usertab, on='UserID', how='inner',sort = True,)

In [16]:
# df_inner.shape

In [17]:

usertab['Signup Date'] = pd.to_datetime(usertab['Signup Date'])

In [18]:
vistorlog_filter.isnull().mean()*100

webClientID       0.000000
VisitDateTime     0.000000
ProductID         7.985769
UserID            0.000000
Activity         13.486657
Browser           0.000000
OS                0.000000
dtype: float64

In [19]:
vistorlog_filter.fillna('Missing',inplace=True)

In [20]:
cat_var = [var for var in vistorlog_filter.columns if vistorlog_filter[var].dtype =='O']
for var in cat_var:
    vistorlog_filter[var]=vistorlog_filter[var].apply(lambda x : x.upper())
    

In [21]:
vistorlog_filter['ProductID'] = vistorlog_filter['ProductID'].apply(lambda x : np.nan if x=='MISSING' else x)
vistorlog_filter['Activity'] = vistorlog_filter['Activity'].apply(lambda x : np.nan if x=='MISSING' else x)

In [22]:
vistorlog_filter = vistorlog_filter.sort_values(by=['UserID','VisitDateTime'], ascending=[True,False])
vistorlog_filter.head()

,webClientID,VisitDateTime,ProductID,UserID,Activity,Browser,OS
3452388,WI10000020634,2018-05-18 10:43:56.628,PR100258,U100002,PAGELOAD,CHROME MOBILE,ANDROID
5179952,WI10000020634,2018-05-18 10:39:53.385,PR101097,U100002,NaN,CHROME MOBILE,ANDROID
4655019,WI10000020634,2018-05-18 10:39:46.717,PR100258,U100002,CLICK,CHROME MOBILE,ANDROID
270999,WI10000020634,2018-05-18 10:39:41.834,PR100258,U100002,NaN,CHROME MOBILE,ANDROID
5811371,WI10000020634,2018-05-18 10:35:29.008,PR100258,U100002,PAGELOAD,CHROME MOBILE,ANDROID


In [23]:
vistorlog_filter['Activity'].ffill(inplace =True)

In [24]:
# No_of_days_Visited_7_Days(How many days a user was active on platform in the last 7 days.)

vistorlog_filter['VisitDateTime_only'] = vistorlog_filter['VisitDateTime'].dt.date

In [25]:
def Visited_7_Days(df): 
    df = df.copy()
    enddate = df.VisitDateTime_only.max()
    strdate = df.VisitDateTime_only.max() - timedelta(days=6) 

    return df[(df.VisitDateTime_only >= strdate) & (df.VisitDateTime_only < enddate+timedelta(days=1) )]
    
# Visited_7_Days(df_inner)

In [26]:
def fun_No_of_days_Visited_7_Days( ):

    df1 = Visited_7_Days(vistorlog_filter)
    df1 = df1[['UserID','VisitDateTime_only']]
    df1.drop_duplicates(inplace=True)
    No_of_days_Visited_7_Days = df1.groupby('UserID')['VisitDateTime_only'].count().to_dict()
    return No_of_days_Visited_7_Days
    
# fun_No_of_days_Visited_7_Days()      

In [27]:
# set(df_inner.UserID.values)
vistorlog_filter['No_of_days_Visited_7_Days'] = vistorlog_filter['UserID'].map(fun_No_of_days_Visited_7_Days())
vistorlog_filter['No_of_days_Visited_7_Days'].fillna(0, inplace=True)

In [28]:
# No_Of_Products_Viewed_15_Days(Number of Products viewed by the user in the last 15 days)

def Visited_15_Days(df):
    df = df.copy()
    enddate = df.VisitDateTime_only.max()
    strdate = df.VisitDateTime_only.max() - timedelta(days=14) 
    return df[(df.VisitDateTime_only >= strdate) & (df.VisitDateTime_only < enddate+timedelta(days=1))]

In [29]:
def fun_No_Of_Products_Viewed_15_Days( ):

    df1 = Visited_15_Days(vistorlog_filter)
    df1 = df1[['UserID','ProductID']]
    df1.drop_duplicates(inplace=True)
    No_Of_Products_Viewed_15_Days = df1.groupby('UserID')['ProductID'].count().to_dict()
    return No_Of_Products_Viewed_15_Days

In [30]:
vistorlog_filter['No_Of_Products_Viewed_15_Days'] = vistorlog_filter['UserID'].map(fun_No_Of_Products_Viewed_15_Days())
vistorlog_filter['No_Of_Products_Viewed_15_Days'].fillna(0, inplace=True)



In [31]:
usertab = usertab.sort_values(by='UserID')

In [32]:
# User_Vintage(Vintage (In Days) of the user as of today)

usertab['User_Vintage'] = usertab['Signup Date'].apply(lambda x : (pd.to_datetime('2018-5-28',utc=True)-x))

In [33]:
usertab['User_Vintage'] = usertab['Signup Date'].apply(lambda x : (pd.to_datetime('2018-5-28',utc=True)-x))
usertab['User_Vintage'] = usertab['User_Vintage'].dt.days

In [34]:
# Most_Active_OS(Most Frequently used OS by user. )

# Most_Active_OS = df_inner.groupby(['UserID'])['OS'].agg(pd.Series.mode).to_dict()
# df_inner['Most_Active_OS'] = df_inner['UserID'].map(Most_Active_OS)
# df_inner['Most_Active_OS'].apply(tuple)

In [35]:
active_os_temp = vistorlog_filter[['UserID','OS','Browser']].groupby(['UserID','OS'])['Browser'].count().reset_index().sort_values(by='OS')

#most active os

active_os=active_os_temp.iloc[active_os_temp.groupby('UserID').apply(lambda x: x['Browser'].idxmax())]

#create dic to map

active_os_dic = pd.Series(active_os['OS'].values,index=active_os['UserID']).to_dict()
vistorlog_filter['Most_Active_OS']= vistorlog_filter['UserID'].map(active_os_dic)

In [36]:
# Pageloads_last_7_days (Count of Page loads in the last 7 days by the user)
def fun_Pageloads_last_7_days():
    df1 = Visited_7_Days(vistorlog_filter)
    df1= df1[['UserID','Activity']][df1.Activity =='PAGELOAD']
    return df1.groupby('UserID')['Activity'].count().to_dict()

vistorlog_filter['Pageloads_last_7_days'] = vistorlog_filter['UserID'].map(fun_Pageloads_last_7_days())
vistorlog_filter['Pageloads_last_7_days'].fillna(0,inplace=True)



In [37]:
# Clicks_last_7_days(Count of Clicks in the last 7 days  by the user)
def fun_Clicks_last_7_days():
    df1 = Visited_7_Days(vistorlog_filter)
    df1= df1[['UserID','Activity']][df1.Activity =='CLICK']
    return df1.groupby('UserID')['Activity'].count().to_dict()

vistorlog_filter['Clicks_last_7_days'] = vistorlog_filter['UserID'].map(fun_Clicks_last_7_days())
vistorlog_filter['Clicks_last_7_days'].fillna(0,inplace=True)

In [38]:
# Recently_Viewed_Product('Most recently viewed (page loads) product by the user.
# If a user has not viewed any product then put it as Product101.')
df_product = vistorlog_filter[vistorlog_filter.ProductID.notnull()][vistorlog_filter.Activity =='PAGELOAD'].groupby(['UserID','ProductID'])['VisitDateTime'].max().reset_index().sort_values(by=['UserID','VisitDateTime'], ascending =[True,False])
df_product = df_product.loc[df_product.groupby(['UserID'])['VisitDateTime'].idxmax()]

Recently_Viewed_Product_dict = pd.Series(df_product.ProductID.values,index=df_product.UserID).to_dict()
# Recently_Viewed_Product_dict

In [39]:
vistorlog_filter['Recently_Viewed_Product'] = vistorlog_filter['UserID'].map(Recently_Viewed_Product_dict)
vistorlog_filter['Recently_Viewed_Product'].fillna('Product101',inplace=True)

In [40]:
# Most_Viewed_product_15_Days (Most frequently viewed (page loads) product by the user in the last 15 days. 
#If there are multiple products that have a similar number of page loads then , consider the recent one. 
#If a user has not viewed any product in the last 15 days then put it as Product101. )

In [41]:
def fun_No_Of_Products_Viewed_15_Days( ):
    df1 = Visited_15_Days(vistorlog_filter)
    df1 = df1[['UserID','ProductID','VisitDateTime','Activity']][df1.Activity =='PAGELOAD'][df1.ProductID.notnull()]
#     df1.drop_duplicates(inplace=True)
    df_temp = df1.groupby(['UserID','ProductID']).agg({'ProductID':'count','VisitDateTime':'max'}).rename(columns={'ProductID':'COUNT','VisitDateTime':'MAX_TIME'}).reset_index().sort_values(by=['UserID','COUNT','MAX_TIME'], ascending=[True,False,False])
    df_temp = df_temp.loc[df_temp.groupby(['UserID'])['COUNT'].idxmax()]
    Most_Viewed_product_Product_dict = pd.Series(df_product.ProductID.values,index=df_product.UserID).to_dict()
    return Most_Viewed_product_Product_dict

# fun_No_Of_Products_Viewed_5_Days()

In [42]:
vistorlog_filter['Most_Viewed_product_15_Days'] = vistorlog_filter['UserID'].map(fun_No_Of_Products_Viewed_15_Days())
vistorlog_filter['Most_Viewed_product_15_Days'].fillna('Product101',inplace=True)

In [43]:
df_final = vistorlog_filter[['UserID','No_of_days_Visited_7_Days','No_Of_Products_Viewed_15_Days','Most_Viewed_product_15_Days','Most_Active_OS','Recently_Viewed_Product','Pageloads_last_7_days','Clicks_last_7_days']]

In [44]:
df_final.drop_duplicates(inplace = True)

In [45]:
df_final['User_Vintage'] = usertab['User_Vintage'].values

In [46]:
result = pd.read_csv('sample_submission_M7Vpb9f.csv')
result.columns

Index(['UserID', 'No_of_days_Visited_7_Days', 'No_Of_Products_Viewed_15_Days',
       'User_Vintage', 'Most_Viewed_product_15_Days', 'Most_Active_OS',
       'Recently_Viewed_Product', 'Pageloads_last_7_days',
       'Clicks_last_7_days'],
      dtype='object')

In [47]:
df_final = df_final[['UserID', 'No_of_days_Visited_7_Days', 'No_Of_Products_Viewed_15_Days',
       'User_Vintage', 'Most_Viewed_product_15_Days', 'Most_Active_OS',
       'Recently_Viewed_Product', 'Pageloads_last_7_days',
       'Clicks_last_7_days']]

In [48]:
df_final.head()

,UserID,No_of_days_Visited_7_Days,No_Of_Products_Viewed_15_Days,User_Vintage,Most_Viewed_product_15_Days,Most_Active_OS,Recently_Viewed_Product,Pageloads_last_7_days,Clicks_last_7_days
3452388,U100002,0.0,2.0,52,PR100258,ANDROID,PR100258,0.0,0.0
341562,U100003,1.0,2.0,1020,PR100079,WINDOWS,PR100079,1.0,2.0
2261134,U100004,1.0,15.0,340,PR100069,WINDOWS,PR100069,1.0,0.0
1765326,U100005,1.0,3.0,680,PR100234,ANDROID,PR100234,2.0,0.0
5832241,U100006,1.0,1.0,54,PR101111,ANDROID,PR101111,1.0,0.0


In [49]:
df_final.shape

(34050, 9)

In [50]:
df_final.to_csv('sample_submission_8.csv', index=False)